In [ ]:
#Old plot for efficient frontier
expected_returns = combined_returns_cleaned.mean()
covariance_matrix = combined_returns_cleaned.cov()
risk_free_rate_cleaned
# Extract the  daily risk-free rate from 24.10.2022
risk_free_rate_one = risk_free_rate_cleaned.loc['2022-10-24', 'Daily_Risk_Free_Rate']
#??? Is the daily_risk_free rate correct? Also should not I use different risk free rate than the last one? I should work with daily log returns right?
print(risk_free_rate_one)
#risk_free_rate_cleaned.loc[risk_free_rate_cleaned['Date'] == '24.10.2022', 'Daily_Risk_Free_Rate'].iloc[0]

# Assuming 'expected_returns', 'covariance_matrix', and 'risk_free_rate_one' are defined
def portfolio_stats(weights, expected_returns, covariance_matrix, timeAnnual):
    """
    Calculate portfolio return and portfolio standard deviation.
    """
    # Annualized return
    portfolio_return = np.sum(weights * expected_returns) * timeAnnual
    # Annualized standard deviation
    portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights))) * np.sqrt(timeAnnual)
    return portfolio_return, portfolio_stddev

# Simulate random portfolios
num_portfolios = 100000
timeAnnual = 252  # Trading year
results = np.zeros((3, num_portfolios))  # Rows: return, risk (std dev), Sharpe ratio
for i in range(num_portfolios):
    # Generate random weights
    weights = np.random.random(len(expected_returns))
    weights /= np.sum(weights)  # Normalize weights to sum to 1
    # Use the unified portfolio_stats function
    portfolio_return, portfolio_stddev = portfolio_stats(weights, expected_returns, covariance_matrix, timeAnnual)
    # Store results
    results[0, i] = portfolio_return  # Return
    results[1, i] = portfolio_stddev  # Risk
    results[2, i] = (portfolio_return - risk_free_rate_one) / portfolio_stddev  # Sharpe ratio
# Print results
print(results)

def minimize_volatility(target_return, num_assets):
    """
    Minimize portfolio volatility for a given target return.
    """
    # Constraints: weights sum to 1 and the portfolio return equals the target return
    constraints = (
        {"type": "eq", "fun": lambda w: np.sum(w) - 1},  # Weights sum to 1
        {"type": "eq", "fun": lambda w: np.sum(w * expected_returns) * timeAnnual - target_return},  # Target return
    )
    # Bounds: weights between 0 and 1
    bounds = tuple((0, 1) for * in range(num*assets))

    # Minimize standard deviation
    result = minimize(
        fun=lambda w: portfolio_stats(w, expected_returns, covariance_matrix, timeAnnual)[1],  # Portfolio standard deviation
        x0=num_assets * [1.0 / num_assets],  # Initial guess: equally weighted portfolio
        args=(),  # No extra arguments are needed
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
    )

    if not result.success:
        raise ValueError("Optimization failed: " + result.message)

    return result.x

# Generate points on the efficient frontier
target_returns = np.linspace(min(expected_returns) * timeAnnual, max(expected_returns) * timeAnnual, 100)
print(target_returns)
frontier_returns = []
frontier_risks = []
for target_return in target_returns:
    weights = minimize_volatility(target_return,8)
    ret, risk = portfolio_stats(weights,expected_returns,covariance_matrix, 252)
    frontier_returns.append(ret)
    frontier_risks.append(risk)
# Plot the efficient frontier
plt.figure(figsize=(10, 6))
plt.scatter(results[1, :], results[0, :], c=results[2, :], cmap="viridis", alpha=0.5)
plt.plot(x, y, color='red', label="Capital Market Line (CML)")
plt.plot(frontier_risks, frontier_returns, color="blue", linewidth=2, label="Efficient Frontier")
plt.title("Efficient Frontier with Classic Parabola")
plt.xlabel("Risk (Standard Deviation)")
plt.ylabel("Expected Return")
plt.colorbar(label="Sharpe Ratio")
plt.legend()
plt.show()

And computation of sharpe

max_sharpe_idx = np.argmax(results[2,:])
max_sharpe_return = results[0, max_sharpe_idx]
max_sharpe_stddev = results[1, max_sharpe_idx]
# Plot CML
x = np.linspace(0, max_sharpe_stddev, 100)
y = risk_free_rate_one + (max_sharpe_return - risk_free_rate_one) / max_sharpe_stddev * x
plt.plot(x, y, color='red', label="Capital Market Line (CML)")
plt.scatter(results[1,:], results[0,:], c=results[2,:], cmap='viridis')
plt.title("Efficient Frontier with Capital Market Line (CML)")
plt.xlabel("Risk (Standard Deviation)")
plt.ylabel("Expected Return")
plt.colorbar(label='Sharpe Ratio')
plt.legend(loc="upper left")
plt.show()